In [ ]:
# init repo notebook
!git clone https://github.com/rramosp/ppdl.git > /dev/null 2> /dev/null
!mv -n ppdl/content/init.py ppdl/content/local . 2> /dev/null
!pip install -r ppdl/content/requirements.txt > /dev/null

# LAB 1. TFP Distributions

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats
from scipy.integrate import quad
from progressbar import progressbar as pbar
from rlxutils import subplots, copy_func
import pandas as pd
import seaborn as sns
import tensorflow as tf
import tensorflow_probability as tfp
tfd = tfp.distributions
tfb = tfp.bijectors

%matplotlib inline

## Task 1: Compute pdf for batch/event shapes

**Without** using Tensorflow. 

**Suggesstion**: Use `scipy.sstats.normal`

In [83]:
# Initialize a single 3-variate Gaussian.
batch_size = np.random.randint((5,2))+2
event_size = np.random.randint(5)+2

batch_size = 3
event_size = 2
mu = np.random.random(size=(batch_size,event_size)).astype(np.float32)

cov = np.random.random((event_size, event_size)).astype(np.float32)*.8+.1
# ensures simetric positive definite for cholesky decomposition to work
cov = cov.dot(cov.T)
        
print (cov)
mvn = tfd.MultivariateNormalTriL(
    loc=mu,
    scale_tril=tf.linalg.cholesky(cov)
)

mvn

In [84]:
s = mvn.sample(5)
s.shape

In [85]:
mvn.log_prob(s)

## Task 2: Batch distributions

the probability for a person to default on a credit depends on his income, and it is modeled with an exponential distribution depending on the person's age $x_i$, older people have flatter exponentials, meaning that as you get older the probability of default for higher incomes is higher than we you are younger.

Observe how we want to **assign a distribution to each data point**


Make a function that, given a set of $x\in\mathbb{R}^n$, returns a batched distribution of `batch_size`$=n$ that assigns to each $x_i$ a $\mathcal{N}(\mu_i, \sigma_i)$ so that:

- $\mu_i = 2x_i$
- $\sigma_i = \text{softplus}(2x_i/10) = \log(e^{x_i/10}+1)$


In [4]:
x = np.linspace(18,60,100)
d = tfd.Exponential(rate=1)

In [8]:
s = d.sample(100000).numpy()
print (s.shape)
plt.hist(s, bins=100, density=True);

## Task 3: Multivariate batch distributions

now each data point is described by more variables, not only the person's age, we add

- the probability of keeping its job depends the number of years of experience. 


## Task 4: Bijector probability

Compute the probability of a scale transformation by hand

$$
\begin{align}
X &\sim \mathcal{Beta}(a,b)\\
Y & = f(x) = kX\\
    \log P_y(Y) &= \log P_x(f^{-1}(Y)) + \log \text{det}\Big[\frac{\partial}{\partial y} f^{-1}(y) \Big] \\
\end{align}
$$

And in this case, since $Y$ is a scalar (single variable distribution)

$$
\begin{align}
f^{-1}(Y) &= Y/k\\
\text{det}\Big[\frac{\partial}{\partial y} f^{-1}(y) \Big] &= \frac{\partial}{\partial y} f^{-1}(y) = 1/k
\end{align}
$$

you **CANNOT** user Tensorflow

In [185]:
a = np.random.random()*1.5+1
b = a + np.random.random()-.5
k = np.random.random()*10

scale = tfb.Scale(k)
d     = scale(tfd.Beta(a,b))  

In [186]:
import ppdl
ppdl.plot_pdf(d)

In [190]:
x = d.sample(10).numpy()

In [191]:
d.log_prob(x).numpy()